In [1]:
# following code is working fine can you make changes to make it faster



%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
wd.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")



wd.get("https://fmbp.aiou.edu.pk/admin/index.php")

txt_username = wd.find_element(By.ID, "txt_username")
txt_password = wd.find_element(By.ID, "txt_password")

txt_username.send_keys("qwert")
txt_password.send_keys("qwert")

wd.save_screenshot("a.png")

login = wd.find_element(By.XPATH, '//*[@id="LLForm"]/button')

login.click()

wd.get("https://fmbp.aiou.edu.pk/admin/application-search.php")

wd.find_element(By.XPATH,"//select[@name='cmb_programme_group']/option[text()='ODL - SSC / M-TECH']").click()

wd.find_element(By.XPATH,"//select[@name='cmb_programe']/option[text()='Matric(General)']").click()

wd.find_element(By.XPATH,"//select[@name='cmb_status']/option[text()='Verified']").click()

wd.find_element(By.XPATH,"//select[@name='cmb_session']/option[text()='Autumn 2022']").click()

search = wd.find_element(By.XPATH, '//*[@id="LLForm"]/div/div[2]/div/div/div[2]/div[17]/div/button[2]')
search.click()

wd.save_screenshot("screenshot.png")

def parse_page(page):
  result=[]
  url=f"https://fmbp.aiou.edu.pk/admin/application-search-result.php?curPage={page}"
  wd.get(url)
  table =  wd.find_element(by=By.XPATH,value='//*[@id="DataTables_Table_0"]')
  tr =  table.find_elements(by=By.TAG_NAME,value='tr')
  for x in range(1, len(tr)):
    elem=tr[x]
    std= [x.text.replace("\n", " ").replace(",", " ") for x in elem.find_elements(by=By.TAG_NAME,value='td')]
    application_ =elem.find_elements(by=By.TAG_NAME,value='td')[1]
    std.append(application_.find_element(by=By.TAG_NAME,value='a').get_attribute("href"))
    result.append(std)
  for y in range(0,len(result)):
    wd.get(result[y][-1])
    result[y][-2]=wd.find_element(by=By.XPATH,value='//*[@id="contact-1"]/ul/li[4]').text.split("\n")[-1]
    result[y]=','.join(result[y])
    print(result[y])
  return result
    

final_result=[]
final_result.append("Serial,ApplicationID,Name,UserName,ProgramGroup,Program,Session,ApplicationStatus,Phone,Link")
start_page=1
end_page=1
for x in range(start_page,end_page+1):
  result=parse_page(x)
  final_result+=result

file_name=f"/content/drive/MyDrive/aiou/data_{start_page}_{end_page}.csv"
with open(file_name, 'w') as f:
    f.write('\n'.join(final_result)

print(final_result[1])